In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession 
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [4]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# Path to the dataset
path = 'dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}')




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 192, 192, 3)          0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, 192, 192, 3)          7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, 192, 192, 3)          0         ['normalization[0][0]']

In [5]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20


179/179 [==============================] - ETA: 0s - loss: 1.6188 - accuracy: 0.4219INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


179/179 [==============================] - 1243s 7s/step - loss: 1.6188 - accuracy: 0.4219 - val_loss: 1.5839 - val_accuracy: 0.4510
Epoch 2/20
179/179 [==============================] - ETA: 0s - loss: 1.3128 - accuracy: 0.5323INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


179/179 [==============================] - 976s 5s/step - loss: 1.3128 - accuracy: 0.5323 - val_loss: 1.4913 - val_accuracy: 0.4827
Epoch 3/20
179/179 [==============================] - ETA: 0s - loss: 1.1095 - accuracy: 0.6168INFO:tensorflow:Assets written to: efficientnet_model\model03\assets


...

179/179 [==============================] - 708s 4s/step - loss: 0.2401 - accuracy: 0.9181 - val_loss: 2.0927 - val_accuracy: 0.5519
Epoch 14/20
179/179 [==============================] - ETA: 0s - loss: 0.2118 - accuracy: 0.9279INFO:tensorflow:Assets written to: efficientnet_model\model14\assets


INFO:tensorflow:Assets written to: efficientnet_model\model14\assets


179/179 [==============================] - 538s 3s/step - loss: 0.2118 - accuracy: 0.9279 - val_loss: 2.0678 - val_accuracy: 0.5579
Epoch 15/20
179/179 [==============================] - ETA: 0s - loss: 0.2072 - accuracy: 0.9263INFO:tensorflow:Assets written to: efficientnet_model\model15\assets


INFO:tensorflow:Assets written to: efficientnet_model\model15\assets


179/179 [==============================] - 578s 3s/step - loss: 0.2072 - accuracy: 0.9263 - val_loss: 2.0787 - val_accuracy: 0.5608
Epoch 16/20
137/179 [=====================>........] - ETA: 1:28 - loss: 0.1986 - accuracy: 0.9357

In [ ]:
test_path = 'dataset/test/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

8/8 [==============================] - 7s 837ms/step - loss: 1.4427 - accuracy: 0.6564
Test Loss: 1.4427
Test Accuracy: 0.6564


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB4
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

# Path to the dataset
path = 'dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # EfficientNetB0 input size

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Train-validation split
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.15, random_state=42)

# Load EfficientNetB0 pre-trained on ImageNet without top layer
base_model = EfficientNetB4(weights='imagenet', include_top=False, input_shape=img_size + (3,))

# Add custom top layers for skin diseases classification
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)  # Add dropout layer
output = Dense(num_classes, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()

# ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint('efficientnet_model/model{epoch:02d}')

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 192, 192, 3)]        0         []                            
                                                                                                  
 rescaling_6 (Rescaling)     (None, 192, 192, 3)          0         ['input_4[0][0]']             
                                                                                                  
 normalization_3 (Normaliza  (None, 192, 192, 3)          7         ['rescaling_6[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_7 (Rescaling)     (None, 192, 192, 3)          0         ['normalization_3[0][0]'

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_val, y_val), callbacks=[checkpoint_callback])

Epoch 1/20


18/18 [==============================] - ETA: 0s - loss: 2.1586 - accuracy: 0.1955INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


INFO:tensorflow:Assets written to: efficientnet_model\model01\assets


18/18 [==============================] - 244s 12s/step - loss: 2.1586 - accuracy: 0.1955 - val_loss: 1.7597 - val_accuracy: 0.3725
Epoch 2/20
18/18 [==============================] - ETA: 0s - loss: 1.8115 - accuracy: 0.3578INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


INFO:tensorflow:Assets written to: efficientnet_model\model02\assets


...

18/18 [==============================] - 139s 8s/step - loss: 0.4877 - accuracy: 0.8429 - val_loss: 1.2547 - val_accuracy: 0.6078
Epoch 19/20
18/18 [==============================] - ETA: 0s - loss: 0.4469 - accuracy: 0.8517INFO:tensorflow:Assets written to: efficientnet_model\model19\assets


INFO:tensorflow:Assets written to: efficientnet_model\model19\assets


18/18 [==============================] - 136s 8s/step - loss: 0.4469 - accuracy: 0.8517 - val_loss: 1.2840 - val_accuracy: 0.6078
Epoch 20/20
18/18 [==============================] - ETA: 0s - loss: 0.3657 - accuracy: 0.8848INFO:tensorflow:Assets written to: efficientnet_model\model20\assets


INFO:tensorflow:Assets written to: efficientnet_model\model20\assets


18/18 [==============================] - 100s 6s/step - loss: 0.3657 - accuracy: 0.8848 - val_loss: 1.2517 - val_accuracy: 0.6275


In [ ]:
test_path = 'dataset/test/'

# Load and preprocess test data
test_labels = []
test_images = []

class_names = sorted(os.listdir(test_path))
num_classes = len(class_names)

for cl in class_names:
    for img in os.listdir(test_path + cl):
        label = np.zeros(num_classes)
        label[class_names.index(cl)] = 1
        test_labels.append(label)
        
        image = cv2.imread(test_path + cl + '/' + img, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (img_size[1], img_size[0]))
        test_images.append(image)

# Convert to numpy arrays
test_labels = np.asarray(test_labels)
test_images = np.asarray(test_images)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

5/5 [==============================] - 7s 1s/step - loss: 1.7025 - accuracy: 0.4903
Test Loss: 1.7025
Test Accuracy: 0.4903


In [ ]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Path to the dataset
path = 'dataset/train/'
class_names = sorted(os.listdir(path))
num_classes = len(class_names)

# Image size
img_size = (192, 192)  # Set the desired input size for images

# Load and preprocess data
labels = []
images = []

for cl in class_names:
    for img in os.listdir(path + cl):
        label = class_names.index(cl)  # Assign a numerical label to each class
        labels.append(label)
        
        image = cv2.imread(path + cl + '/' + img, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
        image = cv2.resize(image, (img_size[1], img_size[0]))
        images.append(image)

# Convert to numpy arrays
labels = np.asarray(labels)
images = np.asarray(images)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Flatten the images array
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

# Initialize the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(params = {
    'tree_method': 'gpu_hist',  # Use GPU accelerated histogram method
    'predictor': 'gpu_predictor',  # Use GPU accelerated prediction
    # Add other parameters as needed
})

# Train the classifier
xgb_classifier.fit(X_train_flat, y_train)

# Predict on the test set
y_pred = xgb_classifier.predict(X_test_flat)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)


c:\Users\surya\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [21:41:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "params" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.3111111111111111
